In [17]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [18]:
# Function to preprocess the data
def preprocess_data_full(df):
    # Extract relevant columns
    df = df[['SEAMAN NAME', 'SEAMAN CODE', 'SEAFARER CODE', 'RANK', 'VESSEL', 'UMUR', 'CERTIFICATE']]
    
    # Handling UMUR: Extract only the numerical part of the age
    df['UMUR'] = df['UMUR'].apply(lambda x: int(str(x).split()[0]))
    
    # Encode categorical columns using Label Encoding
    le_rank = LabelEncoder()
    le_vessel = LabelEncoder()
    le_certificate = LabelEncoder()
    
    df['RANK_ENCODED'] = le_rank.fit_transform(df['RANK'])
    df['VESSEL_ENCODED'] = le_vessel.fit_transform(df['VESSEL'])
    df['CERTIFICATE_ENCODED'] = le_certificate.fit_transform(df['CERTIFICATE'])
    
    # Combine the encoded features
    df_encoded = df[['RANK_ENCODED', 'VESSEL_ENCODED', 'UMUR', 'CERTIFICATE_ENCODED']]
    
    return df, df_encoded, le_rank, le_vessel, le_certificate

# Function to search for candidates and prioritize based on rank, certificate, and vessel
def search_candidates_with_full_details(vessel, rank, certificate, age_range, processed_dfs_full):
    # Determine the correct sheet based on the vessel name prefix
    sheet_code = vessel.split('.')[0]
    if sheet_code in processed_dfs_full:
        df_original, df_encoded, le_rank, le_vessel, le_certificate = processed_dfs_full[sheet_code]
    else:
        return f"Vessel code {sheet_code} does not match any known sheets."

    # Get the VESSEL GROUP ID
    vessel_group_id = df_original[df_original['VESSEL'] == vessel]['VESSEL_ENCODED'].values[0]

    # Filter candidates based on the UMUR range
    df_filtered_indices = df_encoded[(df_encoded['UMUR'] >= age_range[0]) & (df_encoded['UMUR'] <= age_range[1])].index
    
    if df_filtered_indices.empty:
        return "No candidates found within the specified UMUR range."
    
    # Encode user input for cosine similarity comparison
    input_encoded = np.array([
        le_rank.transform([rank])[0],
        vessel_group_id,
        np.mean(age_range),  # Use the average of the UMUR range
        le_certificate.transform([certificate])[0]
    ]).reshape(1, -1)
    
    # Calculate cosine similarity
    similarity_scores = cosine_similarity(input_encoded, df_encoded.loc[df_filtered_indices])
    
    # Combine the original data with similarity scores
    df_filtered = df_original.loc[df_filtered_indices].copy()
    df_filtered['Similarity'] = similarity_scores[0]
    
    # Prioritize candidates by Rank, then Certificate, then Vessel
    df_filtered['Rank_Priority'] = df_filtered['RANK'] == rank
    df_filtered['Certificate_Priority'] = df_filtered['CERTIFICATE'] == certificate
    df_filtered['Vessel_Priority'] = df_filtered['VESSEL'] == vessel
    
    df_filtered = df_filtered.sort_values(
        by=['Rank_Priority', 'Certificate_Priority', 'Vessel_Priority', 'Similarity'],
        ascending=[False, False, False, False]
    )
    
    # Select the top 5 candidates
    result = df_filtered[['SEAMAN NAME', 'SEAMAN CODE', 'SEAFARER CODE', 'RANK', 'VESSEL', 'UMUR', 'CERTIFICATE', 'Similarity']].head(5)
    
    # Display the result in a proper format
    return result

# Load your Excel file
file_path = 'data\Seamen Report.xlsx'
xls = pd.ExcelFile(file_path)

In [19]:
# Load the relevant sheets into dataframes
sheets_to_load = ['BC', 'MT', 'TB', 'TK']
dfs = {sheet: pd.read_excel(xls, sheet_name=sheet) for sheet in sheets_to_load}

# Preprocess each sheet
processed_dfs_full = {}
for sheet, df in dfs.items():
    processed_dfs_full[sheet] = preprocess_data_full(df)

C:\Users\ilham\AppData\Local\Temp\ipykernel_22708\3647146866.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['UMUR'] = df['UMUR'].apply(lambda x: int(str(x).split()[0]))
C:\Users\ilham\AppData\Local\Temp\ipykernel_22708\3647146866.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RANK_ENCODED'] = le_rank.fit_transform(df['RANK'])
C:\Users\ilham\AppData\Local\Temp\ipykernel_22708\3647146866.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [24]:
# Usage example
vessel = 'BC. ANGSA LAUT'
rank = 'JURU MUDI'
certificate = 'ANT-D'
age_range = (45, 65)

# Search for candidates
result = search_candidates_with_full_details(vessel, rank, certificate, age_range, processed_dfs_full)
print(result)

                     SEAMAN NAME  SEAMAN CODE  SEAFARER CODE       RANK  \
46                       MUZAINI     20080225     6200351159  JURU MUDI   
101                      KUSNADI     20050301     6200043665  JURU MUDI   
21   FREDERICK NINGKA TANGKUDUNG     20040174     6201005651     SERANG   
69             ALEXANDER KAPITAN     20050450     6200092098     SERANG   
0                    HAMID HALIK     20040312     6200064037    NAKHODA   

                   VESSEL  UMUR CERTIFICATE  Similarity  
46            BC. EPSILON    47       ANT-D    0.994380  
101     BC. SURABAYA RAYA    51       ANT-D    0.973452  
21   BC. BANJARMASIN RAYA    64       ANT-D    0.992496  
69      BC. KAIMANA INDAH    61       ANT-D    0.983849  
0          BC. ANGSA LAUT    57      ANT-IV    0.991439  
